### Create new

In [ ]:

from xen.training.SequenceVAETrainer import SequenceVAETrainer
from xen.data.Filter import NameFilter, SongDataFilter

# paths = ["../../../ai/trainingdata/music/mutopia_guitar/"]
paths = ["/Volumes/Rob/projects/ml_training/music/songsterr/lorna_shore"]
filter = SongDataFilter(instrumentName=NameFilter(exclude=["Percussion", "Bass"]), 
                        timeSignature='4/4', 
                        minNotesPerSequence=4)

trainer = SequenceVAETrainer(modelPath="../models", modelName="note-lorna-b16")
trainer.loadSongDataset(paths, filter=filter, ticksPerQuarter=4, quartersPerMeasure=4, measuresPerSequence=1)
trainer.createModel(latentDim = 3, hiddenLayers = 2)



### Load Existing

In [ ]:

from xen.training.SequenceVAETrainer import SequenceVAETrainer

trainer = SequenceVAETrainer(modelPath="../models", modelName="mutopia_guitar_16")
trainer.loadSongDataset(["../../../ai/trainingdata/music/mutopia_guitar/"], timesig='4/4', ticksPerQuarter=4, quartersPerMeasure=4, measuresPerSequence=1)
trainer.loadModel()


### Train

In [ ]:
trainer.train(batchSize = 32, epochs = 5000, learningRate = 0.005, patience = 20)
trainer.saveModel(quantize = None)

In [ ]:
trainer.calcRecall()

### Visualise Latent Space

In [ ]:
trainer.plotLatentSpace()

In [ ]:
trainer.plotInputOutputSequence(1000, threshold = 0.5)

In [ ]:
trainer.plotOutputValueDistribution()